<a href="https://colab.research.google.com/github/Nickeniklas/stockdata_forecasting/blob/main/stockdata_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock data Forecasting

Using SARIMAX

Fetching with yfinance

